# EcoHome Energy Advisor - Agent Demo

This notebook demonstrates the EcoHome Energy Advisor agent with various real-world scenarios.

## Overview
The Energy Advisor helps homeowners:
- Optimize energy usage and reduce costs
- Maximize solar power utilization
- Get personalized recommendations based on weather and pricing
- Calculate potential savings from energy-efficient practices


## 1. Setup and Initialization


In [1]:
from agent import Agent
from datetime import datetime
from langchain_core.messages import AIMessage


In [ ]:
# Define the system prompt for the Energy Advisor
ECOHOME_SYSTEM_PROMPT = """You are the EcoHome Energy Advisor, an AI assistant specialized in helping homeowners optimize their energy usage, reduce costs, and maximize the benefits of solar power.

Your primary goals are:
1. Help users minimize electricity costs by leveraging time-of-use pricing
2. Maximize the use of solar-generated electricity
3. Provide actionable recommendations for energy optimization
4. Calculate potential savings from energy-efficient practices

Available Tools:
- get_weather_forecast: Get weather forecasts including solar irradiance data
- get_electricity_prices: Get time-of-use electricity pricing by hour
- query_energy_usage: Query historical energy consumption data
- query_solar_generation: Query historical solar generation data
- get_recent_energy_summary: Get summary of recent energy usage and generation
- search_energy_tips: Search for energy-saving tips and best practices
- calculate_energy_savings: Calculate potential savings from optimizations. IMPORTANT: When users ask about savings from reducing usage (e.g., "if I reduce X by Y%, how much would I save?"), you MUST:
  1. First use query_energy_usage to get current usage data for the device/type
  2. Calculate the optimized usage based on the reduction percentage
  3. Then use calculate_energy_savings tool with the current usage, optimized usage, and device type
  4. Present the calculated savings to the user

Guidelines:
- Always use relevant tools to gather current data before making recommendations
- Consider weather forecasts when recommending solar-dependent activities
- Factor in electricity pricing (peak/off-peak) when suggesting optimal times
- Provide specific time recommendations when relevant
- Include cost calculations and savings estimates when possible
- Be clear, concise, and actionable in your responses
- If you need location information, ask the user or use the context provided
- Always explain your reasoning behind recommendations
- When users ask about potential savings from reducing energy usage, you MUST use the calculate_energy_savings tool to provide accurate calculations

When making recommendations:
- Prioritize using solar power when available
- Suggest off-peak hours for high-energy activities when solar isn't available
- Consider both cost savings and environmental impact
- Provide specific time windows for optimal actions

For savings calculations:
- If a user asks "how much would I save if I reduce [device] usage by X%", you MUST:
  1. Query current energy usage for that device type
  2. Calculate the reduced usage (current * (1 - X/100))
  3. Use calculate_energy_savings tool with these values
  4. Report the savings including monthly and annual projections
"""


In [3]:
# Initialize the agent
ecohome_agent = Agent(
    instructions=ECOHOME_SYSTEM_PROMPT,
)

print("✅ Energy Advisor agent initialized successfully!")
print(f"Available tools: {', '.join(ecohome_agent.get_agent_tools())}")


✅ Energy Advisor agent initialized successfully!
Available tools: get_weather_forecast, get_electricity_prices, query_energy_usage, query_solar_generation, get_recent_energy_summary, search_energy_tips, calculate_energy_savings


## 2. Helper Functions


In [4]:
def run_scenario(question, context=None, scenario_name=None):
    """
    Run a scenario with the Energy Advisor and display results.
    
    Args:
        question: The user's question
        context: Optional context (e.g., location)
        scenario_name: Optional name for the scenario
    """
    print("=" * 80)
    if scenario_name:
        print(f"SCENARIO: {scenario_name}")
    else:
        print("SCENARIO")
    print("=" * 80)
    print(f"\n📍 Context: {context if context else 'None'}")
    print(f"❓ Question: {question}\n")
    print("-" * 80)
    
    try:
        # Get response from agent
        response = ecohome_agent.invoke(question=question, context=context)
        
        # Extract and display the final response
        if isinstance(response, dict) and "messages" in response:
            messages = response["messages"]
            if messages:
                final_message = messages[-1]
                print("\n💡 Advisor Response:\n")
                print(final_message.content)
                
                # Show tools used
                print("\n" + "-" * 80)
                print("🔧 Tools Used:")
                tools_used = []
                for msg in messages:
                    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
                        for tool_call in msg.tool_calls:
                            tool_name = tool_call.get("name", "unknown")
                            if tool_name not in tools_used:
                                tools_used.append(tool_name)
                
                if tools_used:
                    for tool in tools_used:
                        print(f"  • {tool}")
                else:
                    print("  (No tools were used)")
        else:
            print("\n💡 Advisor Response:\n")
            print(str(response))
            
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
    
    print("\n" + "=" * 80 + "\n")


## 3. Example Scenarios

Let's run the Energy Advisor through various real-world scenarios to see how it helps optimize energy usage.


### Scenario 1: EV Charging Optimization


In [5]:
run_scenario(
    question="When should I charge my electric car tomorrow to minimize cost and maximize solar power?",
    context="Location: San Francisco, CA",
    scenario_name="EV Charging Optimization"
)


SCENARIO: EV Charging Optimization

📍 Context: Location: San Francisco, CA
❓ Question: When should I charge my electric car tomorrow to minimize cost and maximize solar power?

--------------------------------------------------------------------------------

💡 Advisor Response:

To minimize costs and maximize the use of solar power while charging your electric car tomorrow (October 7, 2023), here are the optimal charging times based on the weather forecast and electricity pricing:

### Weather Forecast for October 7, 2023
- **Sunny conditions** are expected throughout the day, with solar irradiance peaking between 9 AM and 3 PM.
- **Peak solar generation** is expected around:
  - **11 AM**: 647.2 W/m²
  - **12 PM**: 349.4 W/m²
  - **1 PM**: 417.5 W/m²
  - **2 PM**: 501.6 W/m²

### Electricity Pricing
- **Off-Peak Rates** (lower cost):
  - **12 AM - 6 AM**: $0.1293 to $0.1204 per kWh
  - **10 PM - 12 AM**: $0.119 to $0.1111 per kWh
- **Peak Rates** (higher cost):
  - **6 AM - 10 PM**: R

### Scenario 2: Appliance Scheduling


In [6]:
run_scenario(
    question="When should I run my dishwasher and washing machine to save the most money?",
    context="Location: San Francisco, CA",
    scenario_name="Appliance Scheduling"
)


SCENARIO: Appliance Scheduling

📍 Context: Location: San Francisco, CA
❓ Question: When should I run my dishwasher and washing machine to save the most money?

--------------------------------------------------------------------------------

💡 Advisor Response:

To save the most money while running your dishwasher and washing machine in San Francisco, you should consider both the time-of-use electricity pricing and the availability of solar power.

### Electricity Pricing Overview
- **Off-Peak Rates**: 
  - 12 AM - 6 AM: $0.126 - $0.1115 per kWh
  - 10 PM - 12 AM: $0.1153 - $0.1101 per kWh
- **Peak Rates**: 
  - 6 AM - 10 PM: $0.1714 - $0.243 per kWh

### Recommendations
1. **Run During Off-Peak Hours**:
   - **Best Time**: Between **12 AM and 6 AM** or **10 PM and 12 AM**. 
   - This is when the rates are the lowest, ranging from **$0.1101 to $0.126** per kWh.

2. **Maximize Solar Power**:
   - If you have solar panels, consider running your appliances during the day when solar genera

### Scenario 3: Thermostat Optimization


In [7]:
run_scenario(
    question="What's the optimal thermostat setting for today to balance comfort and energy costs?",
    context="Location: San Francisco, CA",
    scenario_name="Thermostat Optimization"
)


SCENARIO: Thermostat Optimization

📍 Context: Location: San Francisco, CA
❓ Question: What's the optimal thermostat setting for today to balance comfort and energy costs?

--------------------------------------------------------------------------------

💡 Advisor Response:

To determine the optimal thermostat setting for today in San Francisco, we need to consider the current weather conditions, the forecast for the day, and the electricity pricing.

### Current Weather Conditions:
- **Current Temperature**: 18°C (64°F)
- **Condition**: Cloudy
- **Forecast High**: 23°C (73°F) around 3 PM
- **Forecast Low**: 10.5°C (50.9°F) overnight

### Electricity Pricing:
- **Peak Hours**: 6 AM - 8 PM (Rates range from $0.1706 to $0.2472 per kWh)
- **Off-Peak Hours**: 12 AM - 5 AM, 9 PM - 11 PM (Rates range from $0.1115 to $0.1225 per kWh)

### Recommendations:
1. **Daytime Thermostat Setting**: 
   - Set your thermostat to **22°C (72°F)** during the day when the temperature is expected to rise to 2

### Scenario 4: Solar Power Maximization


In [8]:
run_scenario(
    question="How can I maximize my use of solar power tomorrow?",
    context="Location: San Francisco, CA",
    scenario_name="Solar Power Maximization"
)


SCENARIO: Solar Power Maximization

📍 Context: Location: San Francisco, CA
❓ Question: How can I maximize my use of solar power tomorrow?

--------------------------------------------------------------------------------

💡 Advisor Response:

To maximize your use of solar power tomorrow in San Francisco, here are some actionable recommendations based on the weather forecast and electricity pricing:

### Weather Forecast for Tomorrow (October 7, 2023)
- **Morning (6 AM - 12 PM)**: 
  - **6 AM**: Sunny, solar irradiance: 0.0
  - **7 AM**: Partly cloudy, solar irradiance: 73.44 W/m²
  - **8 AM**: Partly cloudy, solar irradiance: 102.42 W/m²
  - **9 AM**: Cloudy, solar irradiance: 191.64 W/m²
  - **10 AM**: Partly cloudy, solar irradiance: 139.86 W/m²
  - **11 AM**: Partly cloudy, solar irradiance: 179.28 W/m²
  - **12 PM**: Partly cloudy, solar irradiance: 335.52 W/m² (Peak solar generation)

- **Afternoon (1 PM - 5 PM)**:
  - **1 PM**: Partly cloudy, solar irradiance: 312.0 W/m²
  - **2 P

### Scenario 5: Energy Usage Analysis


In [9]:
run_scenario(
    question="How much did I spend on electricity in the last 24 hours and what was my biggest energy expense?",
    context="Location: San Francisco, CA",
    scenario_name="Energy Usage Analysis"
)


SCENARIO: Energy Usage Analysis

📍 Context: Location: San Francisco, CA
❓ Question: How much did I spend on electricity in the last 24 hours and what was my biggest energy expense?

--------------------------------------------------------------------------------

💡 Advisor Response:

In the last 24 hours, you spent a total of **$24.32** on electricity. 

Your biggest energy expense was from your **Electric Vehicle (EV)**, which consumed **143.72 kWh** and cost you **$18.03**. 

Here's a breakdown of your energy expenses:
- **EV**: $18.03
- **HVAC**: $3.85
- **Appliances**: $2.44

If you have any further questions or need assistance with optimizing your energy usage, feel free to ask!

--------------------------------------------------------------------------------
🔧 Tools Used:
  • get_recent_energy_summary




### Scenario 6: Energy-Saving Tips


In [10]:
run_scenario(
    question="What are the best ways to reduce my HVAC energy consumption?",
    context="Location: San Francisco, CA",
    scenario_name="Energy-Saving Tips"
)


SCENARIO: Energy-Saving Tips

📍 Context: Location: San Francisco, CA
❓ Question: What are the best ways to reduce my HVAC energy consumption?

--------------------------------------------------------------------------------


c:\Users\sebas\Documents\Projects\UDACITY_AgenticAI_withLangChainAndLanGraph\Project 2 - Energy Advisor\tools.py:382: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4



💡 Advisor Response:

Here are some effective ways to reduce your HVAC energy consumption:

1. **Smart Thermostat Usage**: Install a smart thermostat that can automatically adjust temperatures based on your schedule. Set your thermostat to 68°F in winter and 78°F in summer to optimize energy savings. This can save you up to 10% on heating and cooling costs.

2. **Regular Maintenance**: Change air filters monthly to ensure better airflow and efficiency. Clean the outdoor condenser coils annually to maintain optimal performance.

3. **Seal Air Leaks**: Check for and seal air leaks around windows and doors using weatherstripping or caulk. This helps maintain the desired temperature inside your home without overworking your HVAC system.

4. **Insulation**: Add insulation to attics and walls to reduce heating and cooling needs. Proper insulation can significantly lower your energy consumption.

5. **Use Ceiling Fans**: Ceiling fans can help circulate air, making rooms feel cooler. This allo

### Scenario 7: Savings Calculation


In [11]:
run_scenario(
    question="If I reduce my air conditioning usage by 30%, how much money would I save per month?",
    context="Location: San Francisco, CA",
    scenario_name="Savings Calculation"
)


SCENARIO: Savings Calculation

📍 Context: Location: San Francisco, CA
❓ Question: If I reduce my air conditioning usage by 30%, how much money would I save per month?

--------------------------------------------------------------------------------

💡 Advisor Response:

To calculate your potential savings from reducing air conditioning usage by 30%, I need to know your current monthly energy usage for air conditioning in kilowatt-hours (kWh) and the current price per kWh. 

If you have that information, please provide it. If not, I can look up your recent energy usage data to estimate it. Would you like me to do that?

--------------------------------------------------------------------------------
🔧 Tools Used:
  (No tools were used)




### Scenario 8: Solar Generation Analysis


In [12]:
run_scenario(
    question="How much solar power did I generate in the last week and how does it compare to my usage?",
    context="Location: San Francisco, CA",
    scenario_name="Solar Generation Analysis"
)


SCENARIO: Solar Generation Analysis

📍 Context: Location: San Francisco, CA
❓ Question: How much solar power did I generate in the last week and how does it compare to my usage?

--------------------------------------------------------------------------------

💡 Advisor Response:

It appears that there is no recorded solar power generation or energy usage data for the week of October 9 to October 16, 2023. This could be due to a variety of reasons, such as a lack of data collection or system issues.

To better assist you, could you please confirm if your solar system was operational during that time, or if you would like to check a different date range?

--------------------------------------------------------------------------------
🔧 Tools Used:
  • query_solar_generation
  • query_energy_usage




### Scenario 9: Comprehensive Optimization Plan


In [13]:
run_scenario(
    question="Give me a comprehensive plan to optimize my energy usage for the next 3 days considering weather and pricing.",
    context="Location: San Francisco, CA",
    scenario_name="Comprehensive Optimization Plan"
)


SCENARIO: Comprehensive Optimization Plan

📍 Context: Location: San Francisco, CA
❓ Question: Give me a comprehensive plan to optimize my energy usage for the next 3 days considering weather and pricing.

--------------------------------------------------------------------------------

💡 Advisor Response:

Here's a comprehensive energy optimization plan for the next three days in San Francisco, CA, considering the weather forecast and electricity pricing.

### Weather Forecast (Next 3 Days)
- **Day 1**: Partly cloudy with peak solar irradiance around 293 W/m² at 1 PM.
- **Day 2**: Similar conditions with peak solar irradiance around 280 W/m² at 1 PM.
- **Day 3**: Partly cloudy with peak solar irradiance around 300 W/m² at 1 PM.

### Electricity Pricing (Time-of-Use)
- **Off-Peak Rates**: 
  - 12 AM - 6 AM: $0.1136 - $0.1221
  - 10 PM - 12 AM: $0.1213 - $0.1157
- **Peak Rates**: 
  - 6 AM - 10 PM: $0.1716 - $0.2498

### Actionable Recommendations

#### Day 1 (November 9)
- **Morning (6 

### Scenario 10: Quick EV Charging Question


In [14]:
run_scenario(
    question="I need to charge my EV from 20% to 80% battery. When is the cheapest time to do this today?",
    context="Location: San Francisco, CA",
    scenario_name="Quick EV Charging Question"
)


SCENARIO: Quick EV Charging Question

📍 Context: Location: San Francisco, CA
❓ Question: I need to charge my EV from 20% to 80% battery. When is the cheapest time to do this today?

--------------------------------------------------------------------------------

💡 Advisor Response:

To charge your EV from 20% to 80%, you'll want to do it during the cheapest electricity rates today. Here are the time slots with the lowest rates:

- **Off-Peak Hours**:
  - 0:00 AM - 1:00 AM: $0.1117 per kWh
  - 1:00 AM - 2:00 AM: $0.1138 per kWh
  - 2:00 AM - 3:00 AM: $0.1248 per kWh
  - 3:00 AM - 4:00 AM: $0.1158 per kWh
  - 4:00 AM - 5:00 AM: $0.1191 per kWh
  - 5:00 AM - 6:00 AM: $0.1274 per kWh
  - 10:00 PM - 11:00 PM: $0.1761 per kWh
  - 11:00 PM - 12:00 AM: $0.1212 per kWh
  - 12:00 AM - 1:00 AM: $0.1193 per kWh

### Recommendation:
- **Best Time to Charge**: Start charging your EV at **12:00 AM** (midnight) to take advantage of the lowest rate of **$0.1117 per kWh**. 

### Charging Calculation:
T

## 4. Summary

The Energy Advisor agent successfully demonstrates its ability to:
- ✅ Provide personalized recommendations based on weather and pricing data
- ✅ Optimize energy usage timing to minimize costs
- ✅ Maximize solar power utilization
- ✅ Calculate potential savings
- ✅ Offer actionable energy-saving tips
- ✅ Analyze historical energy data

The agent uses various tools including weather forecasts, electricity pricing, energy usage queries, and energy-saving tips to provide comprehensive and actionable advice.
